<a href="https://colab.research.google.com/github/stella-depuydt/CS-167/blob/main/Copy_of_Day03_P2_Missing_Data_Normalization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CS167: Day03 
## Part 2: Missing Data and Normalization

#### CS167: Machine Learning, J-Term 2023

Thursday, January 5th, 2023 -- Session II (10:20-11:35)

[⏮ Day03 Part 1](https://github.com/merriekay/j23_cs167_notes/blob/main/Day03_P1_kNN.ipynb) | [Day04 Part 1⏩]()

## Helpful Links:
📆 [Course Schedule](https://docs.google.com/spreadsheets/d/e/2PACX-1vStj3FCEJqloUMLn2VtHa4yy1ILY6WvABhu4jd4cVUpPGkrx1mEjfTFmd77DMESR9HJ-8UBxgMDJL06/pubhtml?gid=0&single=true) | 🙋[PollEverywhere](https://pollev.com/meredithmoore011) | 📜 [Syllabus](https://analytics.drake.edu/~moore/j23_cs167/Syllabus.html)


# Admin Stuff

You should be working on:
- Notebook #1, is due tonight by 11:59 pm. 
    - to submit, copy and paste the link to your github repository to Blackboard. My link would look like: https://github.com/j23-cs167/notebook-1-python-pandas-and-pokemon-merriekay, but yours will have your GitHub username rather than 'merriekay'
- Notebook #2: is released today, but will be due on Friday 1/6 by 11:59pm.
- Heads up that Quiz #1 will be released on Friday after class and will be due Monday by 11:59pm. 


## Other Things to follow up on:
- `iloc` v `loc` and exclusive v inclusive explanation
- Course Scheudle
- update office hours times

## Before we get started, let's load in our datasets:

In [2]:
#run this cell if you're using Colab:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#import the data:
#make sure the path on the line below corresponds to the path where you put your dataset.
import pandas as pd
path = 'datasets/titanic.csv'
titanic = pd.read_csv('/content/drive/MyDrive/Datasets/titanic.csv')
titanic.head()


,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


# ✨ New Material

# Missing Data:
Most datasets you will work with will not be in perfect shape--you'll need to "clean" the data before you can run any machine learning algorithms on it.

Missing data is a pretty common thing--so much so that there's a special value for missing data: `NaN`, or not a number.

The steps of cleaning data normally include:
1. Identifying which columns have missing data
2. Determining how much data is missing in each column
3. Deciding what to do with the missing data: drop it, fill it, let it be

Notice, in the `deck` column, there are 3 instances of `NaN` we can see...

But what about the other 800 or so rows? Do we have to go through and find them manually? Gross.

In [ ]:
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


## Step 1: Identify Missing Data

In order to ID missing data, we will use a combination of three pandas functions: 
- `isna()`, `notna()`, and `any()` 

## Using `isna()` and `notna()` to find missing data:
- `isna()` will return a boolean series where it is True if the element is `NaN'.
- `notna()` will return a bollean seires where it is True if the element is __not__ `NaN`.


In [ ]:
%%html
<iframe src="https://pandas.pydata.org/docs/reference/api/pandas.isna.html" width="1000" height="500"></iframe>

Let's call `isna()` on the first 5 row of Titanic, and see what we get as an output:

In [ ]:
titanic.loc[:4].isna()
#look at the 'deck' column... 
#identifies where missing data is

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False


That's pretty nifty, but there's gotta be a better way of summarizing this...

# `any()`

In [ ]:
%%html
<iframe src="https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.any.html" width="1000" height="350"></iframe>

Let's use `any()` on the call to `isna()` we just did to let us know which columns have missing data:

In [ ]:
titanic.isna().any()

survived       False
pclass         False
sex            False
age             True
sibsp          False
parch          False
fare           False
embarked        True
class          False
who            False
adult_male     False
deck            True
embark_town     True
alive          False
alone          False
dtype: bool

Several columns are missing data: `age`, `embarked`, `deck`, and `embark_town`.

Wouldn't it be great to know how much data is missing in each of those columns?

## Step 2: How much data is missing?
To decide how to handle our missing data, it's important to know how much missing data each column has.
- If the missing data is a small proportion of the data, we choose to drop those rows completely from the dataset. 
- However, if most of the rows are missing data for a specific column, maybe it's a sign that we don't need to use that column.

There are multiple ways of doing this, but one of the quickest/easiest is using `value_counts()`

## `value_counts()`
Great, so now that we know which columns are missing data, let's check to see how much data they are missing using `value_counts()`.

In [ ]:
%%html
<iframe src="https://pandas.pydata.org/docs/reference/api/pandas.Series.value_counts.html" width="1000" height="350"></iframe>

In [4]:
titanic.deck.value_counts(dropna=False) 
#688 missing values

NaN    688
C       59
B       47
D       33
E       32
A       15
F       13
G        4
Name: deck, dtype: int64

In [5]:
titanic.age.value_counts(dropna=False)
#177 missing values

NaN      177
24.00     30
22.00     27
18.00     26
28.00     25
        ... 
36.50      1
55.50      1
0.92       1
23.50      1
74.00      1
Name: age, Length: 89, dtype: int64

In [6]:
titanic.embarked.value_counts(dropna=False)
#2 missing values

S      644
C      168
Q       77
NaN      2
Name: embarked, dtype: int64

In [7]:
titanic.embark_town.value_counts(dropna=False)
#2 missing values

Southampton    644
Cherbourg      168
Queenstown      77
NaN              2
Name: embark_town, dtype: int64

So, here are our results: 

| **Column**    | **Num Rows Missing** |
|:---------------|----------------------|
| `deck`         | 688                  |
| `age`          | 177                  |
| `embarked`    | 2                    |
| `embark_town` | 2                    |

Now with this new information, it's up to us to decide what to do with these missing values

## Step 3: Decide how to handle missing data

There are 3 main options here:
- drop the missing data from the dataset (either col or row)
- fill the missing data with a suitable replacement
- let it be and cross our fingers

### Option 1: Drop it using `dropna()`

If there isn't much missing data, and/or you have a very large dataset, dropping the row that includes the missing data is a viable option.

In [ ]:
print("before: ", titanic.shape)
titanic.dropna()
print("after: " , titanic.shape)

before:  (891, 15)
after:  (891, 15)


huh... that's weird. We know that there's missing data, why didn't the shape change?

Pandas is trying to protect you, and rather than dropping the rows "in place", it is returning a dataframe with the rows dropped--as written, we're just not saving it's return. There are two ways to fix this:
- save what `dropna()` is returning in a variable (see below)
- add the parameter `inplace=True` to the function call, and it will drop the rows in the original dataset (be careful with this one)

In [ ]:
print("before: ", titanic.shape)
no_missing_data = titanic.dropna()
#titanic.dropna(inplace=True)
print("after: " , no_missing_data.shape)

before:  (891, 15)
after:  (182, 15)


That's better, but wow, most of our dataset is gone now if we drop all of the rows that have missing data. If this happens to you, you'll probably want to re-load your data to have the full dataset to work with. 

In [ ]:
# if that happens, you'll want to re-run your data loading code:
path = 'datasets/titanic.csv'
titanic = pd.read_csv(path)

`embarked` and `embark_town` don't have many rows missing... let's use `dropna()` to drop them in place:
- the parameter `subset` allows us to provide a list of columns that we want any missing data to be dropped from.

In [ ]:
print("before: ", titanic.shape)
titanic.dropna(inplace=True, subset=['embarked', 'embark_town'])
print("after: ", titanic.shape)

before:  (891, 15)
after:  (889, 15)


### Option 2:  Fill it using `fillna()`

If dropping all of the data will make your dataset too sparse, consider filling the missing values with something else. 

What do you think we should use to fill in the missing data in the `age` column?
- we probably don't want to throw off our statistics... 

In [ ]:
titanic.head(7)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
5,0,3,male,NaN,0,0,8.4583,Q,Third,man,True,NaN,Queenstown,no,True
6,0,1,male,54.0,0,0,51.8625,S,First,man,True,E,Southampton,no,True


The `fillna()` functiona llows `NaN` values to be filled with a given value like so: 

In [8]:
print("before: ", titanic['age'].isna().any())
age_mean = titanic['age'].mean()
titanic['age'].fillna(age_mean, inplace=True)
print("after: ", titanic['age'].isna().any())
titanic.head(7)

before:  True
after:  False


,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.000000,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.000000,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.000000,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.000000,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.000000,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
5,0,3,male,29.699118,0,0,8.4583,Q,Third,man,True,NaN,Queenstown,no,True
6,0,1,male,54.000000,0,0,51.8625,S,First,man,True,E,Southampton,no,True


## Option #3: Let it be ❄️

What's so bad about missing data? Why do we care if some data is missing?

What happens if we try to do math with `NaN`? Try it out for yourself:

In [ ]:
import numpy as np
a = np.nan

In [ ]:
#try out some addition/subtraction

In [ ]:
#try out some multiplication/division

In [ ]:
#what about taking something to the power? (**)

In [ ]:
# modulo or integer division? (% or //)

In [ ]:
# what about '=='? Is np.nan == np.nan?

In [ ]:
# what happens if you take the average of this list of numbers?
my_series = pd.Series([2,2,3,np.nan,3])
my_series.mean()

2.5

In [ ]:
my_series.median()

2.5

## Summary: Missing Data Functions
- `isna()`: returns True for any missing data
- `notna()`: returns True for any data that is __not__ `NaN`
- `any()`: returns true if any of the elements in a Series is True
- `value_counts()`: returns a list of the values in a Series, use `dropna=False` to see `NaN` values
- `dropna()`: drops rows or columns (specify which axis, 1 or 0) that have missing data. Don't forget to either save the result of the call or add `inplace=True` as a parameter.
- `fillna()`: replaces missing data with a given value (generally 0 or the mean)

# Normalization:

__Normalizing data:__
- rescale attrbute values so they're about the same
- adjusting values measured on different scales to a common scale

## A Simple Normalization:
One simple method of normalizing data is to replace each value with a proportion relativeto the max value. 

For example, the oldest person on the Titanic was 80, so: 

| **age** | **replaced by** |
|---------|:------------------|
| 80      | 80/80 = 1        |
| 50      | 50/80 = 0.625    |
| 48      | 48/80 = 0.6      |
| 25      | 25/80 = 0.3125   |
| 4       | 4/80 = 0.05      |

## After Normalization

<div>
<img src="https://github.com/merriekay/j23_cs167_notes/blob/main/images/day03_norm_dist1.png?raw=1"/>
</div>

### Now is sex over-emphasized?

## Z-Score: Another Normalization Method

__Idea__: rather than normalize to proportion of max, normalize based on how mnay standard deviations they are away from the mean.

__Standard Deviation__: usually represened as $\sigma$ (sigma), a kind of 'average' distance from the average value. 
- a low standard deviation indicates that the values tend to be close to the mean
- a high standard deviation indicates that the values are spread out over a wider range.

## Standard Deviation:
<div>
<img src="https://github.com/merriekay/j23_cs167_notes/blob/main/images/day03_std.png?raw=1" width=600/>
</div>

## Standard Deviation Calculation:

## $\sigma = \sqrt{\frac{1}{N}\sum_{i=1}^{N}(x_i - \mu)^2}$

1. Find the mean, represented as $\mu$ (mu)
2. Then, for each number, subtract the mean and square the result.
3. Then, find the mean of those squared differences. 
4. Take the square root of tht and we are done. 

Let $\mu$ be the mean, then standard deviation of $x_1, x+2, ..., x_N$ is:

## $\sigma = \sqrt{\frac{(x_1-\mu)^2 + (x_2 - \mu)^2+ ... + (x_N-\mu)^2}{N}}$

# Corrected Sample Standard Deviation

The mean of a sample tends to be a good estimate for the mean of the entire population (on average), but.. 
- standard deviation of samples tend to be _smaller_ than the standard deviation of the entier population.

__Bessel's correction__ says that you should divide by $N-1$ instead of N when working with a sample (as we usually do in machine learning tasks), and your estimate will be a little less biased. 

## $\sigma = \sqrt{\frac{(x_1-\mu)^2 + (x_2 - \mu)^2+ ... + (x_N-\mu)^2}{N-1}}$

# Computing the Z-Score
After computing the corrected sample standard deviation,

to normlaize, replace each value $x_i$ with it's Z-Socre based on the mean ($\mu$) and standard deviation ($\sigma$) of it's column. 

## $Z-score: \frac{x_i- \mu}{\sigma}$

## Exampe Z-Score Calculation

For example: 
On the Titanic:
- sex mean(0:male, 1:female): 0.35
- sex standard deviation: 0.48
- age mean: 29.7
- age standard deviation: 13

<div>
<img src="https://github.com/merriekay/j23_cs167_notes/blob/main/images/day03_zscore.png?raw=1" width=600/>
</div>


<div>
<img src="https://github.com/merriekay/j23_cs167_notes/blob/main/images/day03_zscore_ex.png?raw=1" width=600/>
</div>


<div>
<img src="https://github.com/merriekay/j23_cs167_notes/blob/main/images/day03_zscore_improvement.png?raw=1" width=600/>
</div>

# Normalization Code:
Let's try out some code now:



In [9]:
#make sure your data is loaded and ready to go (one of the top few cells)
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


## New function `replace()`

Called on a dataframe, will repalce values given in `to_replace` with `value`. 

Let's use this to make the `sex` column of the dataset numeric.

In [10]:
titanic['sex'] = titanic['sex'].replace(to_replace='female', value=1)
titanic['sex'] = titanic['sex'].replace(to_replace='male', value=0)
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,0,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,1,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,1,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,1,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,0,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


## Calculating z-score:
Now that we have the data as 1s and 0s, let's calculate the mean and standard deviation.

In [11]:
s_mean = titanic.sex.mean()
s_std = titanic.sex.std()

#replace column with each entry's z-score
titanic.sex = (titanic.sex - s_mean)/s_std
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,-0.737281,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,1.354813,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,1.354813,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,1.354813,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,-0.737281,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


Next, you'd need to repeat this process for all of the predictor columns -- so they're all of compareable size. 

## Exercise: 

Normalize each of the predictor columsn in the iris dataset.

> Note: you need a way to transform the new reading (the specimen) that you will make the precition on so that the new one and the training data will all be on the same sclae. How can you do that?

Repeat your kNN prediction code with the normalized data. 
- Does the value of k hchange the predictions? 